# Import

In [1]:
import argparse
import logging
import time
from os.path import exists

import numpy as np
import torch
import torch.nn as nn

import pyro
import pyro.contrib.examples.polyphonic_data_loader as poly
import pyro.distributions as dist
import pyro.poutine as poutine
from pyro.distributions import TransformedDistribution
from pyro.distributions.transforms import affine_autoregressive
from pyro.infer import (
    SVI,
    JitTrace_ELBO,
    Trace_ELBO,
    TraceEnum_ELBO,
    TraceTMC_ELBO,
    config_enumerate,
)
from pyro.optim import ClippedAdam

/Users/joshuadimasaka/miniconda3/envs/opensendaibench/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/joshuadimasaka/miniconda3/envs/opensendaibench/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /Users/joshuadimasaka/miniconda3/envs/opensendaibench/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <8E6D6BF5-9658-33B9-9D3C-DF587B2F99E7> /Users/joshuadimasaka/miniconda3/envs/opensendaibench/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
torch.get_default_device()
import os
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
torch.set_default_device('mps')
torch.get_default_device()

device(type='mps', index=0)

# Argument Parse

In [3]:
assert pyro.__version__.startswith("1.9.1")
n = 10          # number of epochs
lr = 0.0003     # learning rate
b1 = 0.96       # beta1
b2 = 0.999      # beta2
cn = 10.0       # clip norm
lrd = 0.99996   # lr decay
wd = 2.0        # weight decay
mbs = 20        # mini-batch size
ae = 1000       # annealing epochs
maf = 0.2       # minimum annealing epochs
rdr = 0.1       # rnn dropout rate
iafs = 0        # number of iafs
id = 100        # iaf dim
cf = 0          # checkpoint frequency

lopt = ''       # load opt
lmod = ''       # load model
sopt = ''       # save opt
smod = ''       # save model

cuda = False    
jit = False
tmc = False
tmcelbo = False
tmc_num_samples = 10

l = 'dmm_20240110.log'   # log



In [4]:
# setup logging
logging.basicConfig(
    level=logging.DEBUG, format="%(message)s", filename=l, filemode="w"
)
console = logging.StreamHandler()
console.setLevel(logging.INFO)
logging.getLogger("").addHandler(console)

In [5]:
data = poly.load_data(poly.JSB_CHORALES)

training_seq_lengths = data["train"]["sequence_lengths"] 
# gives the size of each of the 229 sequences (called chorales); 
# e.g.. 1st has 129 as length (or the longest chorale)

training_data_sequences = data["train"]["sequences"]
# gives the actual sequences in a form of matrix, where:
# each row is a sequence or a chorale
# each column is a step for each sequence (max of 129)
# each entry for a given pair of row and column has length 88 (piano keys)

test_seq_lengths = data["test"]["sequence_lengths"]
test_data_sequences = data["test"]["sequences"]
val_seq_lengths = data["valid"]["sequence_lengths"]
val_data_sequences = data["valid"]["sequences"]

N_train_data = len(training_seq_lengths)
N_train_time_slices = float(torch.sum(training_seq_lengths))
N_mini_batches = int(
    N_train_data / mbs
    + int(N_train_data % mbs > 0)
)

In [6]:
logging.info(
    "N_train_data: %d     avg. training seq. length: %.2f    N_mini_batches: %d"
    % (N_train_data, training_seq_lengths.float().mean(), N_mini_batches)
)

N_train_data: 229     avg. training seq. length: 60.29    N_mini_batches: 12


In [7]:
# how often we do validation/test evaluation during training
val_test_frequency = 50
# the number of samples we use to do the evaluation
n_eval_samples = 1

In [8]:
# how often we do validation/test evaluation during training
val_test_frequency = 50
# the number of samples we use to do the evaluation
n_eval_samples = 1

# package repeated copies of val/test data for faster evaluation
# (i.e. set us up for vectorization)
def rep(x):
    rep_shape = torch.Size([x.size(0) * n_eval_samples]) + x.size()[1:]
    repeat_dims = [1] * len(x.size())
    repeat_dims[0] = n_eval_samples
    return (
        x.repeat(repeat_dims)
        .reshape(n_eval_samples, -1)
        .transpose(1, 0)
        .reshape(rep_shape)
    )

# get the validation/test data ready for the dmm: pack into sequences, etc.
val_seq_lengths = rep(val_seq_lengths)
test_seq_lengths = rep(test_seq_lengths)

In [9]:
import os
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

def reverse_sequences(mini_batch, seq_lengths):
    reversed_mini_batch = torch.zeros_like(mini_batch)
    for b in range(mini_batch.size(0)):
        T = seq_lengths[b]
        time_slice = torch.arange(T - 1, -1, -1, device=mini_batch.device)
        reversed_sequence = torch.index_select(mini_batch[b, :, :], 0, time_slice)
        reversed_mini_batch[b, 0:T, :] = reversed_sequence
    return reversed_mini_batch

def get_mini_batch_mask(mini_batch, seq_lengths):
    mask = torch.zeros(mini_batch.shape[0:2])
    for b in range(mini_batch.shape[0]):
        mask[b, 0 : seq_lengths[b]] = torch.ones(seq_lengths[b])
    return mask

def get_mini_batch_mps(mini_batch_indices, sequences, seq_lengths, cuda=False, mps=True):

    # get the sequence lengths of the mini-batch
    seq_lengths = seq_lengths[mini_batch_indices]

    # sort the sequence lengths
    # indices should be either on cpu or on the same device as the indexed tensor (cpu)
    _, sorted_seq_length_indices = torch.sort(seq_lengths)
    sorted_seq_length_indices = sorted_seq_length_indices.flip(0)
    sorted_seq_lengths = seq_lengths[sorted_seq_length_indices]
    sorted_mini_batch_indices = mini_batch_indices[sorted_seq_length_indices]

    # compute the length of the longest sequence in the mini-batch
    T_max = torch.max(seq_lengths)
    # this is the sorted mini-batch
    mini_batch = sequences[sorted_mini_batch_indices, 0:T_max, :]
    # this is the sorted mini-batch in reverse temporal order
    mini_batch_reversed = reverse_sequences(mini_batch, sorted_seq_lengths)
    # get mask for mini-batch
    mini_batch_mask = get_mini_batch_mask(mini_batch, sorted_seq_lengths)

    # cuda() here because need to cuda() before packing
    if cuda:
        mini_batch = mini_batch.cuda()
        mini_batch_mask = mini_batch_mask.cuda()
        mini_batch_reversed = mini_batch_reversed.cuda()

    if mps:
        mini_batch = mini_batch.to(torch.device('mps'))
        mini_batch_mask = mini_batch_mask.to(torch.device('mps'))
        mini_batch_reversed = mini_batch_reversed.to(torch.device('mps'))

    # do sequence packing
    mini_batch_reversed = nn.utils.rnn.pack_padded_sequence(
        mini_batch_reversed, sorted_seq_lengths.to('cpu'), batch_first=True
    )
    mini_batch_reversed = mini_batch_reversed.to(torch.device('mps'))

    return mini_batch, mini_batch_reversed, mini_batch_mask, sorted_seq_lengths

In [10]:
(
    val_batch,
    val_batch_reversed,
    val_batch_mask,
    val_seq_lengths,
) = get_mini_batch_mps(
    torch.arange(n_eval_samples * val_data_sequences.shape[0]),
    rep(val_data_sequences).to(torch.device('mps')),
    val_seq_lengths.to(torch.device('mps')),
    cuda=False, mps=True, 
)
(
    test_batch,
    test_batch_reversed,
    test_batch_mask,
    test_seq_lengths,
) = get_mini_batch_mps(
    torch.arange(n_eval_samples * test_data_sequences.shape[0]),
    rep(test_data_sequences).to(torch.device('mps')),
    test_seq_lengths.to(torch.device('mps')),
    cuda=False, mps=True,
)

# Define all the classes now

In [11]:
class Emitter(nn.Module):
    """
    Parameterizes the bernoulli observation likelihood `p(x_t | z_t)`
    """

    def __init__(self, input_dim, z_dim, emission_dim):
        super().__init__()
        # initialize the three linear transformations used in the neural network
        self.lin_z_to_hidden = nn.Linear(z_dim, emission_dim, device=torch.device('mps'))
        self.lin_hidden_to_hidden = nn.Linear(emission_dim, emission_dim, device=torch.device('mps'))
        self.lin_hidden_to_input = nn.Linear(emission_dim, input_dim, device=torch.device('mps'))
        # initialize the two non-linearities used in the neural network
        self.relu = nn.ReLU()

    def forward(self, z_t):
        """
        Given the latent z at a particular time step t we return the vector of
        probabilities `ps` that parameterizes the bernoulli distribution `p(x_t|z_t)`
        """
        h1 = self.relu(self.lin_z_to_hidden(z_t))
        h2 = self.relu(self.lin_hidden_to_hidden(h1))
        ps = torch.sigmoid(self.lin_hidden_to_input(h2))
        return ps

In [12]:
class GatedTransition(nn.Module):
    """
    Parameterizes the gaussian latent transition probability `p(z_t | z_{t-1})`
    See section 5 in the reference for comparison.
    """

    def __init__(self, z_dim, transition_dim):
        super().__init__()
        # initialize the six linear transformations used in the neural network
        self.lin_gate_z_to_hidden = nn.Linear(z_dim, transition_dim, device=torch.device('mps'))
        self.lin_gate_hidden_to_z = nn.Linear(transition_dim, z_dim, device=torch.device('mps'))
        self.lin_proposed_mean_z_to_hidden = nn.Linear(z_dim, transition_dim, device=torch.device('mps'))
        self.lin_proposed_mean_hidden_to_z = nn.Linear(transition_dim, z_dim, device=torch.device('mps'))
        self.lin_sig = nn.Linear(z_dim, z_dim, device=torch.device('mps'))
        self.lin_z_to_loc = nn.Linear(z_dim, z_dim, device=torch.device('mps'))
        # modify the default initialization of lin_z_to_loc
        # so that it's starts out as the identity function
        self.lin_z_to_loc.weight.data = torch.eye(z_dim).to(torch.device('mps'))
        self.lin_z_to_loc.bias.data = torch.zeros(z_dim).to(torch.device('mps'))
        # initialize the three non-linearities used in the neural network
        self.relu = nn.ReLU()
        self.softplus = nn.Softplus()

    def forward(self, z_t_1):
        """
        Given the latent `z_{t-1}` corresponding to the time step t-1
        we return the mean and scale vectors that parameterize the
        (diagonal) gaussian distribution `p(z_t | z_{t-1})`
        """
        # compute the gating function
        _gate = self.relu(self.lin_gate_z_to_hidden(z_t_1))
        gate = torch.sigmoid(self.lin_gate_hidden_to_z(_gate))
        # compute the 'proposed mean'
        _proposed_mean = self.relu(self.lin_proposed_mean_z_to_hidden(z_t_1))
        proposed_mean = self.lin_proposed_mean_hidden_to_z(_proposed_mean)
        # assemble the actual mean used to sample z_t, which mixes a linear transformation
        # of z_{t-1} with the proposed mean modulated by the gating function
        loc = (1 - gate) * self.lin_z_to_loc(z_t_1) + gate * proposed_mean
        # compute the scale used to sample z_t, using the proposed mean from
        # above as input the softplus ensures that scale is positive
        scale = self.softplus(self.lin_sig(self.relu(proposed_mean)))
        # return loc, scale which can be fed into Normal
        return loc, scale

In [13]:
class Combiner(nn.Module):
    """
    Parameterizes `q(z_t | z_{t-1}, x_{t:T})`, which is the basic building block
    of the guide (i.e. the variational distribution). The dependence on `x_{t:T}` is
    through the hidden state of the RNN (see the PyTorch module `rnn` below)
    """

    def __init__(self, z_dim, rnn_dim):
        super().__init__()
        # initialize the three linear transformations used in the neural network
        self.lin_z_to_hidden = nn.Linear(z_dim, rnn_dim, device=torch.device('mps'))
        self.lin_hidden_to_loc = nn.Linear(rnn_dim, z_dim, device=torch.device('mps'))
        self.lin_hidden_to_scale = nn.Linear(rnn_dim, z_dim, device=torch.device('mps'))
        # initialize the two non-linearities used in the neural network
        self.tanh = nn.Tanh()
        self.softplus = nn.Softplus()

    def forward(self, z_t_1, h_rnn):
        """
        Given the latent z at at a particular time step t-1 as well as the hidden
        state of the RNN `h(x_{t:T})` we return the mean and scale vectors that
        parameterize the (diagonal) gaussian distribution `q(z_t | z_{t-1}, x_{t:T})`
        """
        # combine the rnn hidden state with a transformed version of z_t_1
        h_combined = 0.5 * (self.tanh(self.lin_z_to_hidden(z_t_1)) + h_rnn)
        # use the combined hidden state to compute the mean used to sample z_t
        loc = self.lin_hidden_to_loc(h_combined)
        # use the combined hidden state to compute the scale used to sample z_t
        scale = self.softplus(self.lin_hidden_to_scale(h_combined))
        # return loc, scale which can be fed into Normal
        return loc, scale

In [14]:
class DMM(nn.Module):
    """
    This PyTorch Module encapsulates the model as well as the
    variational distribution (the guide) for the Deep Markov Model
    """

    def __init__(
        self,
        input_dim=88,
        z_dim=100,
        emission_dim=100,
        transition_dim=200,
        rnn_dim=600,
        num_layers=1,
        rnn_dropout_rate=0.1, #0.0,
        num_iafs=0,
        iaf_dim=100, #50,
        use_cuda=False,
    ):
        super().__init__()
        # instantiate PyTorch modules used in the model and guide below
        self.emitter = Emitter(input_dim, z_dim, emission_dim)
        self.trans = GatedTransition(z_dim, transition_dim)
        self.combiner = Combiner(z_dim, rnn_dim)
        # dropout just takes effect on inner layers of rnn
        rnn_dropout_rate = 0.0 if num_layers == 1 else rnn_dropout_rate
        self.rnn = nn.RNN(
            input_size=input_dim,
            hidden_size=rnn_dim,
            nonlinearity="relu",
            batch_first=True,
            bidirectional=False,
            num_layers=num_layers,
            dropout=rnn_dropout_rate,
            device=torch.device('mps'),
        )

        # if we're using normalizing flows, instantiate those too
        self.iafs = [
            affine_autoregressive(z_dim, hidden_dims=[iaf_dim]) for _ in range(num_iafs)
        ]
        self.iafs_modules = nn.ModuleList(self.iafs)

        # define a (trainable) parameters z_0 and z_q_0 that help define the probability
        # distributions p(z_1) and q(z_1)
        # (since for t = 1 there are no previous latents to condition on)
        self.z_0 = nn.Parameter(torch.zeros(z_dim).to(torch.device('mps')))
        self.z_q_0 = nn.Parameter(torch.zeros(z_dim).to(torch.device('mps')))
        # define a (trainable) parameter for the initial hidden state of the rnn
        self.h_0 = nn.Parameter(torch.zeros(1, 1, rnn_dim).to(torch.device('mps')))

        self.use_cuda = use_cuda
        # if on gpu cuda-ize all PyTorch (sub)modules
        if use_cuda:
            self.cuda()

    # the model p(x_{1:T} | z_{1:T}) p(z_{1:T})
    def model(
        self,
        mini_batch,
        mini_batch_reversed,
        mini_batch_mask,
        mini_batch_seq_lengths,
        annealing_factor=1.0,
    ):
        # this is the number of time steps we need to process in the mini-batch
        T_max = mini_batch.size(1)

        # register all PyTorch (sub)modules with pyro
        # this needs to happen in both the model and guide
        pyro.module("dmm", self)

        # set z_prev = z_0 to setup the recursive conditioning in p(z_t | z_{t-1})
        z_prev = self.z_0.expand(mini_batch.size(0), self.z_0.size(0))

        # we enclose all the sample statements in the model in a plate.
        # this marks that each datapoint is conditionally independent of the others
        with pyro.plate("z_minibatch", len(mini_batch), device=torch.device('mps')  ):
            # sample the latents z and observed x's one time step at a time
            # we wrap this loop in pyro.markov so that TraceEnum_ELBO can use multiple samples from the guide at each z
            for t in pyro.markov(range(1, T_max + 1)):
                # the next chunk of code samples z_t ~ p(z_t | z_{t-1})
                # note that (both here and elsewhere) we use poutine.scale to take care
                # of KL annealing. we use the mask() method to deal with raggedness
                # in the observed data (i.e. different sequences in the mini-batch
                # have different lengths)

                # first compute the parameters of the diagonal gaussian distribution p(z_t | z_{t-1})
                z_loc, z_scale = self.trans(z_prev)

                # then sample z_t according to dist.Normal(z_loc, z_scale)
                # note that we use the reshape method so that the univariate Normal distribution
                # is treated as a multivariate Normal distribution with a diagonal covariance.
                with poutine.scale(scale=annealing_factor):
                    z_t = pyro.sample(
                        "z_%d" % t,
                        dist.Normal(z_loc.to(torch.device('mps')), z_scale.to(torch.device('mps')))
                        .mask(mini_batch_mask[:, t - 1 : t])
                        .to_event(1),
                    )

                # compute the probabilities that parameterize the bernoulli likelihood
                emission_probs_t = self.emitter(z_t)
                # the next statement instructs pyro to observe x_t according to the
                # bernoulli distribution p(x_t|z_t)
                pyro.sample(
                    "obs_x_%d" % t,
                    dist.Bernoulli(emission_probs_t.to(torch.device('mps')))
                    .mask(mini_batch_mask[:, t - 1 : t])
                    .to_event(1),
                    obs=mini_batch[:, t - 1, :],
                )
                # the latent sampled at this time step will be conditioned upon
                # in the next time step so keep track of it
                z_prev = z_t

    # the guide q(z_{1:T} | x_{1:T}) (i.e. the variational distribution)
    def guide(
        self,
        mini_batch,
        mini_batch_reversed,
        mini_batch_mask,
        mini_batch_seq_lengths,
        annealing_factor=1.0,
    ):
        # this is the number of time steps we need to process in the mini-batch
        T_max = mini_batch.size(1)
        # register all PyTorch (sub)modules with pyro
        pyro.module("dmm", self)

        # if on gpu we need the fully broadcast view of the rnn initial state
        # to be in contiguous gpu memory
        h_0_contig = self.h_0.expand(
            1, mini_batch.size(0), self.rnn.hidden_size
        ).contiguous()
        # push the observed x's through the rnn;
        # rnn_output contains the hidden state at each time step
        rnn_output, _ = self.rnn(mini_batch_reversed, h_0_contig)
        # reverse the time-ordering in the hidden state and un-pack it
        rnn_output = poly.pad_and_reverse(rnn_output, mini_batch_seq_lengths)
        # set z_prev = z_q_0 to setup the recursive conditioning in q(z_t |...)
        z_prev = self.z_q_0.expand(mini_batch.size(0), self.z_q_0.size(0))

        # we enclose all the sample statements in the guide in a plate.
        # this marks that each datapoint is conditionally independent of the others.
        with pyro.plate("z_minibatch", len(mini_batch), device=torch.device('mps') ):
            # sample the latents z one time step at a time
            # we wrap this loop in pyro.markov so that TraceEnum_ELBO can use multiple samples from the guide at each z
            for t in pyro.markov(range(1, T_max + 1)):
                # the next two lines assemble the distribution q(z_t | z_{t-1}, x_{t:T})

                z_loc, z_scale = self.combiner(z_prev, rnn_output[:, t - 1, :])


                # if we are using normalizing flows, we apply the sequence of transformations
                # parameterized by self.iafs to the base distribution defined in the previous line
                # to yield a transformed distribution that we use for q(z_t|...)
                if len(self.iafs) > 0:
                    z_dist = TransformedDistribution(
                        dist.Normal(z_loc.to(torch.device('mps')), z_scale.to(torch.device('mps'))), self.iafs
                    )
                    assert z_dist.event_shape == (self.z_q_0.size(0),)
                    assert z_dist.batch_shape[-1:] == (len(mini_batch),)
                else:
                    z_dist = dist.Normal(z_loc, z_scale)
                    assert z_dist.event_shape == ()
                    assert z_dist.batch_shape[-2:] == (
                        len(mini_batch),
                        self.z_q_0.size(0),
                    )

                # sample z_t from the distribution z_dist
                with pyro.poutine.scale(scale=annealing_factor):
                    if len(self.iafs) > 0:
                        # in output of normalizing flow, all dimensions are correlated (event shape is not empty)
                        z_t = pyro.sample(
                            "z_%d" % t, z_dist.mask(mini_batch_mask[:, t - 1])
                        )
                    else:
                        # when no normalizing flow used, ".to_event(1)" indicates latent dimensions are independent
                        z_t = pyro.sample(
                            "z_%d" % t,
                            z_dist.mask(mini_batch_mask[:, t - 1 : t]).to_event(1),
                        )
                # the latent sampled at this time step will be conditioned upon in the next time step
                # so keep track of it
                z_prev = z_t


# Instantiate the dmm

In [15]:
dmm = DMM(
    rnn_dropout_rate=rdr,
    num_iafs=iafs,
    iaf_dim=id,
    use_cuda=cuda,
).to(device)

# Setup optimizer

In [16]:
adam_params = {
    "lr": lr,
    "betas": (b1, b2),
    "clip_norm": cn,
    "lrd": lrd,
    "weight_decay": wd,
}
adam = ClippedAdam(adam_params)


# Setup inference algorithm

In [17]:
if tmc:
    if jit:
        raise NotImplementedError("no JIT support yet for TMC")
    tmc_loss = TraceTMC_ELBO()
    dmm_guide = config_enumerate(
        dmm.guide,
        default="parallel",
        num_samples=tmc_num_samples,
        expand=False,
    )
    svi = SVI(dmm.model, dmm_guide, adam, loss=tmc_loss)
elif tmcelbo:
    if jit:
        raise NotImplementedError("no JIT support yet for TMC ELBO")
    elbo = TraceEnum_ELBO()
    dmm_guide = config_enumerate(
        dmm.guide,
        default="parallel",
        num_samples=tmc_num_samples,
        expand=False,
    )
    svi = SVI(dmm.model, dmm_guide, adam, loss=elbo)
else:
    elbo = JitTrace_ELBO() if jit else Trace_ELBO()
    svi = SVI(dmm.model, dmm.guide, adam, loss=elbo)

# Now we're going to define some functions we need to form the main training loop

## Saves the model and optimizer states to disk

In [18]:
def save_checkpoint():
    logging.info("saving model to %s..." % smod)
    torch.save(dmm.state_dict(), smod)
    logging.info("saving optimizer states to %s..." % sopt)
    adam.save(sopt)
    logging.info("done saving model and optimizer checkpoints to disk.")

# Loads the model and optimizer states from disk

In [19]:
def load_checkpoint():
    assert exists(lopt) and exists(
        lmod
    ), "--load-model and/or --load-opt misspecified"
    logging.info("loading model from %s..." % lmod)
    dmm.load_state_dict(torch.load(lmod, weights_only=False))
    logging.info("loading optimizer states from %s..." % lopt)
    adam.load(lopt)
    logging.info("done loading model and optimizer states.")

# Prepare a mini-batch and take a gradient step to minimize -elbo

In [20]:
# JD - need to modify this to use MPS
import os
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

def process_minibatch(epoch, which_mini_batch, shuffled_indices):
    if ae > 0 and epoch < ae:
        # compute the KL annealing factor approriate for the current mini-batch in the current epoch
        min_af = maf
        annealing_factor = min_af + (1.0 - min_af) * (
            float(which_mini_batch + epoch * N_mini_batches + 1)
            / float(ae * N_mini_batches)
        )
    else:
        # by default the KL annealing factor is unity
        annealing_factor = 1.0

    # compute which sequences in the training set we should grab
    mini_batch_start = which_mini_batch * mbs
    mini_batch_end = np.min(
        [(which_mini_batch + 1) * mbs, N_train_data]
    )
    mini_batch_indices = shuffled_indices[mini_batch_start:mini_batch_end]
    # grab a fully prepped mini-batch using the helper function in the data loader
    (
        mini_batch,
        mini_batch_reversed,
        mini_batch_mask,
        mini_batch_seq_lengths,
    ) = get_mini_batch_mps(
        mini_batch_indices,
        training_data_sequences.to(torch.device('mps')),
        training_seq_lengths.to(torch.device('mps')),
        cuda=False, mps=True,
    )

    # do an actual gradient step
    loss = svi.step(
        mini_batch,
        mini_batch_reversed,
        mini_batch_mask,
        mini_batch_seq_lengths,
        annealing_factor,
    )
    # keep track of the training loss
    return loss

# Helper function for doing evaluation

In [21]:
def do_evaluation():
    # put the RNN into evaluation mode (i.e. turn off drop-out if applicable)
    dmm.rnn.eval()

    # compute the validation and test loss n_samples many times
    val_nll = svi.evaluate_loss(
        val_batch, val_batch_reversed, val_batch_mask, val_seq_lengths
    ) / float(torch.sum(val_seq_lengths))
    test_nll = svi.evaluate_loss(
        test_batch, test_batch_reversed, test_batch_mask, test_seq_lengths
    ) / float(torch.sum(test_seq_lengths))

    # put the RNN back into training mode (i.e. turn on drop-out if applicable)
    dmm.rnn.train()
    return val_nll, test_nll

# if checkpoint files provided, load model and optimizer states from disk before we start training
if lopt != "" and lmod != "":
    load_checkpoint()

# Training Loop

In [22]:
times = [time.time()]
for epoch in range(n):
    # if specified, save model and optimizer states to disk every checkpoint_freq epochs
    if cf > 0 and epoch > 0 and epoch % cf == 0:
        save_checkpoint()

    # accumulator for our estimate of the negative log likelihood (or rather -elbo) for this epoch
    epoch_nll = 0.0
    # prepare mini-batch subsampling indices for this epoch
    shuffled_indices = torch.randperm(N_train_data)

    # process each mini-batch; this is where we take gradddient steps
    for which_mini_batch in range(N_mini_batches):
        # print(which_mini_batch)
        epoch_nll += process_minibatch(epoch, which_mini_batch, shuffled_indices)

    # report training diagnostics
    times.append(time.time())
    epoch_time = times[-1] - times[-2]
    logging.info(
        "[training epoch %04d]  %.4f \t\t\t\t(dt = %.3f sec)"
        % (epoch, epoch_nll / N_train_time_slices, epoch_time)
    )

    # do evaluation on test and validation data and report results
    if val_test_frequency > 0 and epoch > 0 and epoch % val_test_frequency == 0:
        val_nll, test_nll = do_evaluation()
        logging.info(
            "[val/test epoch %04d]  %.4f  %.4f" % (epoch, val_nll, test_nll)
        )

[training epoch 0000]  62.9117 				(dt = 23.994 sec)
[training epoch 0001]  57.9943 				(dt = 20.723 sec)
[training epoch 0002]  41.6934 				(dt = 21.538 sec)
[training epoch 0003]  23.1886 				(dt = 20.541 sec)
[training epoch 0004]  17.2338 				(dt = 21.395 sec)
[training epoch 0005]  14.7713 				(dt = 21.725 sec)
[training epoch 0006]  13.3703 				(dt = 22.322 sec)
[training epoch 0007]  12.4769 				(dt = 22.209 sec)
[training epoch 0008]  12.0307 				(dt = 20.665 sec)
[training epoch 0009]  11.8544 				(dt = 20.997 sec)


# Conclusion: It looks like the GPU using Pyro is not effective at all. Using CPU gives roughly 3.5 seconds per epoch whereas using GPU gives ~22 seconds per epoch. I don't know perhaps Pyro is more suitable for CUDA and not MPS. I will try to see if NumPyro is better because it has the same set of nice examples of Deep Markov Model, and also uses JAX.